In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
from wordcloud import WordCloud

Импортирую таблицу, которую ранее успешно спарсил

In [34]:
df = pd.read_csv('Таблица.csv')
df

Срок размещения объявления          Модель машины  Год выпуска  \
0                 5 часов назад          BMW X7 4.4 AT         2024   
1                  4 часа назад     BMW 7 серия 3.0 AT         2023   
2                 5 часов назад          BMW X7 3.0 AT         2024   
3                   1 час назад          BMW XM 4.4 AT         2023   
4                  5 дней назад          BMW X7 3.0 AT         2020   
...                         ...                    ...          ...   
4993             1 неделю назад  BMW 6 серия GT 2.0 AT         2020   
4994                4 дня назад     BMW 3 серия 2.0 AT         2014   
4995             1 неделю назад     BMW 4 серия 2.0 AT         2014   
4996             1 неделю назад     BMW 3 серия 2.0 AT         2019   
4997             1 неделю назад     BMW 5 серия 3.0 AT         2019   

     Пробег, км  Цена машины, ₽   Тип машины  Привод Тип бензина  \
0            1         21900000  внедорожник  полный      бензин   
1           50         17000000        седан  полный      дизель   
2            1         16900000  внедорожник  полный      дизель   
3           15         25490000  внедорожник  полный      гибрид   
4       65 000         10190000  внедорожник  полный      дизель   
...         ...             ...          ...     ...         ...   
4993    83 252          5399000      лифтбек  полный      дизель   
4994   139 000          2155000        седан  полный      дизель   
4995    83 549          2919700         купе  полный      дизель   
4996    44 260          3877140        седан  задний      бензин   
4997    95 405          4650000        седан  полный      дизель   

                                                 Ссылка  
0     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
1     https://www.avito.ru/moskva/avtomobili/bmw_7_s...  
2     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
3     https://www.avito.ru/moskva/avtomobili/bmw_xm_...  
4     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
...                                                 ...  
4993  https://www.avito.ru/moskva/avtomobili/bmw_6_s...  
4994  https://www.avito.ru/moskva/avtomobili/bmw_3_s...  
4995  https://www.avito.ru/moskva/avtomobili/bmw_4_s...  
4996  https://www.avito.ru/moskva/avtomobili/bmw_3_s...  
4997  https://www.avito.ru/moskva/avtomobili/bmw_5_s...  

[4998 rows x 9 columns]

### Сначала преобразуем данные в числовой формат

In [35]:
for index, value in df["Пробег, км"].items():
    if isinstance(value, int):
        a = value
    else:
        a = int(value.replace('\xa0', ''))
    df.loc[index, "Пробег, км"] = a
    
for index, value in df["Цена машины, ₽"].items():
    if isinstance(value, int):
        b = value
    else:
        b = int(value.replace('\xa0', ''))
    df.loc[index, "Цена машины, ₽"] = b

### Необходимо поработать со сроком размещения объявления

###### Для начала узнает количество уникальных элементов в df["Срок размещения объявления"]

In [36]:
unique = {}
for index, i in df["Срок размещения объявления"].items():
    i = i.split(" ")
    if i[1] not in unique.keys():
        unique[i[1]] = 1
    else:
        unique[i[1]] += 1
    if i[1] == "марта":
        print(i)
print(unique)

['29', 'марта', '18:27']
{'часов': 436, 'часа': 384, 'час': 127, 'дней': 834, 'день': 903, 'дня': 1439, 'минуты': 29, 'неделю': 677, 'минут': 19, 'недели': 147, 'секунд': 2, 'марта': 1}


In [37]:
for index, i in df["Срок размещения объявления"].items():
    s = i.split(" ")
    if s[1] == "час" or s[1] == "часов" or s[1] == "часа" or s[1] == "минуты" or s[1] == "минута" or s[1] == "минут" or s[1] == "секунды" or s[1] == "секунда" or s[1] == "секунд":
        a = 0
    elif s[1] == "день" or s[1] == "дня" or s[1] == "дней":
        a = int(s[0])
    elif s[1] == "неделю" or s[1] == "недели" or s[1] == "недель":
        a = int(s[0]) * 7
    elif s[1] == "месяц" or s[1] == "месяца" or s[1] == "месяцев":
        a == int(s[0]) * 31 
    else:
        a = "больше месяца назад"
        
    if a != "больше месяца назад":
        df.loc[index, "Срок размещения объявления"] = int(a)
    else:
        df.drop(index, inplace=True)

In [38]:
df = df.rename(columns={'Срок размещения объявления': 'Срок размещения объявления (дней)'})
df

Срок размещения объявления (дней)          Модель машины  Год выпуска  \
0                                    0          BMW X7 4.4 AT         2024   
1                                    0     BMW 7 серия 3.0 AT         2023   
2                                    0          BMW X7 3.0 AT         2024   
3                                    0          BMW XM 4.4 AT         2023   
4                                    5          BMW X7 3.0 AT         2020   
...                                ...                    ...          ...   
4993                                 7  BMW 6 серия GT 2.0 AT         2020   
4994                                 4     BMW 3 серия 2.0 AT         2014   
4995                                 7     BMW 4 серия 2.0 AT         2014   
4996                                 7     BMW 3 серия 2.0 AT         2019   
4997                                 7     BMW 5 серия 3.0 AT         2019   

     Пробег, км  Цена машины, ₽   Тип машины  Привод Тип бензина  \
0             1        21900000  внедорожник  полный      бензин   
1            50        17000000        седан  полный      дизель   
2             1        16900000  внедорожник  полный      дизель   
3            15        25490000  внедорожник  полный      гибрид   
4         65000        10190000  внедорожник  полный      дизель   
...         ...             ...          ...     ...         ...   
4993      83252         5399000      лифтбек  полный      дизель   
4994     139000         2155000        седан  полный      дизель   
4995      83549         2919700         купе  полный      дизель   
4996      44260         3877140        седан  задний      бензин   
4997      95405         4650000        седан  полный      дизель   

                                                 Ссылка  
0     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
1     https://www.avito.ru/moskva/avtomobili/bmw_7_s...  
2     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
3     https://www.avito.ru/moskva/avtomobili/bmw_xm_...  
4     https://www.avito.ru/moskva/avtomobili/bmw_x7_...  
...                                                 ...  
4993  https://www.avito.ru/moskva/avtomobili/bmw_6_s...  
4994  https://www.avito.ru/moskva/avtomobili/bmw_3_s...  
4995  https://www.avito.ru/moskva/avtomobili/bmw_4_s...  
4996  https://www.avito.ru/moskva/avtomobili/bmw_3_s...  
4997  https://www.avito.ru/moskva/avtomobili/bmw_5_s...  

[4997 rows x 9 columns]

1) Интенсивность использования авто (Износ). Будет считаться так:  $ \frac {Износ_-авто}{Максимальный_-износ_-авто_-в_-выборке} $ . Где Износ_авто будет считаться так: $ {(текущий_-год\  -  \ год_-выпуска_-авто) * пробег_-авто}$ - если пробег не равен 0. И так: $ {(текущий_-год\  -  \ год_-выпуска_-авто)} * 100 $ - если пробег равен 0. Умножение на 100 за год необходимо из-за амортизации машины.

In [ ]:
iznoc0 = []
iznoc1 = []
for index, i in df.iterrows():
    if i["Пробег, км"] != 0:
        iznoc0.append((2024 - i["Год выпуска"])*i["Пробег, км"])
    elif i["Пробег, км"] == 0:
        iznoc0.append((2024 - i["Год выпуска"])*100)
        
a = max(iznoc0)

for i in iznoc0:
    iznoc1.append(round(i/a, 5)) #до 5-ти знаков, чтобы не было букв в числах
    
df["Износ"] = iznoc1


1) Состояние автомобиля (новый/б/у): эта характеристика позволит покупателям лучше понимать общее состояние автомобиля и его историю.
2) Серия Бмв (ну там X, M и тд)
3) Лакшери сегмент / простой / семейный и тд

2. Наглядное представление того, была ли машина в употребленнии ранее или нет. Этот признак будет полезен покупателям на авито, тк не все продавцы добросовестные и не пишут информацию об этом в объявлениях, иначе это было бы спарс

In [ ]:
df["Машина б/у"] = df["Пробег, км"] > 0